In [1]:
!pip install scikit-fuzzy

     |████████████████████████████████| 1.0MB 1.5MB/s eta 0:00:01
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-cp37-none-any.whl size=894070 sha256=7ca28103e84d4a5d57ae8d3be7061819999b1d68ce5b59242b3b6682774d3971
  Stored in directory: /home/shatil/.cache/pip/wheels/b9/4e/77/da79b16f64ef1738d95486e2731eea09d73e90a72465096600
Successfully built scikit-fuzzy


In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class FuzzyPy():
    # Defines TRiangular membership finction f:x->y, with 'a' and 'c' the base of triangle and 'b' is peak
    def trimf(self, x, a, b, c):
        X1 = (x - a) / (b - a)
        X2 = (c - x) / (c - b)
        X3 = np.minimum(X1, X2)
        X4 = np.zeros(x.size)
        y = np.maximum(X3, X4)
        return y

    # Defines Trapezoidal membership finction f:x->y, with 'a' and 'd' the base of trpezoid and 'b' and 'c' the shoulder
    def trapmf(self, x, a, b, c, d):
        X1 = (x - a) / (b - a)
        X2 = np.ones(x.size)
        X3 = (d - x) / (d - c)
        X4 = np.minimum(np.minimum(X1, X2), X3)
        X5 = np.zeros(x.size)
        y = np.maximum(X4, X5)
        return y

    def gaussmf(self, x, c, v):
        """Compute Gaussian Membership function. """
        y = [np.exp(-np.power((i - c) , 2) / (2 * v ** 2.0)) for i in x]
        return y
    
    def softmax(x):
        """Compute softmax values for each sets of scores in x."""
        return np.exp(x)/np.sum(np.exp(x), axis=0)
    

class Triangle(FuzzyPy):
    def __init__(self, x, low, middle, high):
        self.low = self.trimf(x, low[0], low[1], low[2])
        self.middle = self.trimf(x, middle[0], middle[1], middle[2])
        self.high = self.trimf(x, high[0], high[1], high[2])

class Triangle5(FuzzyPy):
    def __init__(self, x, low2,low1, middle, high1,high2):
        self.low2 = self.trimf(x, low2[0], low2[1], low2[2])
        self.low1 = self.trimf(x, low1[0], low1[1], low1[2])
        self.middle = self.trimf(x, middle[0], middle[1], middle[2])
        self.high1 = self.trimf(x, high1[0], high1[1], high1[2])
        self.high2 = self.trimf(x, high2[0], high2[1], high2[2])

class Trapezoid(FuzzyPy):
    def __init__(self, x, low, middle, high):
        self.low = self.trapmf(x, low[0], low[1], low[2], low[3])
        self.middle = self.trapmf(x, middle[0], middle[1], middle[2], middle[3])
        self.high = self.trapmf(x, high[0], high[1], high[2], high[3])

class Trapezoid5(FuzzyPy):
    def __init__(self, x, low2,low1, middle, high1,high2):
        self.low2 = self.trapmf(x, low2[0], low2[1], low2[2], low2[3])
        self.low1 = self.trapmf(x, low1[0], low1[1], low1[2], low1[3])
        self.middle = self.trapmf(x, middle[0], middle[1], middle[2], middle[3])
        self.high1 = self.trapmf(x, high1[0], high1[1], high1[2], high1[3])
        self.high2 = self.trapmf(x, high2[0], high2[1], high2[2], high2[3])

class Gauss(FuzzyPy):
    def __init__(self, x, low, middle, high):
        self.low = self.gaussmf(x, low[0], low[1])
        self.middle = self.gaussmf(x, middle[0], middle[1])
        self.high = self.gaussmf(x, high[0], high[1])

class Gauss5(FuzzyPy):
    def __init__(self, x, low2,low1, middle, high1,high2):
        self.low2 = self.gaussmf(x, low2[0], low2[1])
        self.low1 = self.gaussmf(x, low1[0], low1[1])
        self.middle = self.gaussmf(x, middle[0], middle[1])
        self.high1 = self.gaussmf(x, high1[0], high1[1])
        self.high2 = self.gaussmf(x, high2[0], high2[1])
        
  ## Operators Defined

def intersect(A,B):
     """Intersect two membership functions"""
     return np.minimum(A,B)


def union(A, B):
    """Union of two membership functions"""
    return np.maximum(A, B)

def complement(A):
    """complement of membership"""
    return 1-A

def alphaCut(A,a):
    """Alpha cut on membership function"""
    from copy import deepcopy
    B = deepcopy(A)
    B[(B < a)]= 0
    return B

def add(A,B):
    """Adds two fuzzy membership functions/sets"""
    return np.minimum(A+B,1)


def sub(A,B):
    """Subtracts two fuzzy membership functions/sets"""
    return np.maximum(A-B,0)

In [4]:
def defuzz(x, mfx):
   
    sum_moment_area = 0.0
    sum_area = 0.0

    # If the membership function is a singleton fuzzy set:
    if len(x) == 1:
        return x[0]*mfx[0] / np.fmax(mfx[0], np.finfo(float).eps).astype(float)

    # else return the sum of moment*area/sum of area
    for i in range(1, len(x)):
        x1 = x[i - 1]
        x2 = x[i]
        y1 = mfx[i - 1]
        y2 = mfx[i]

        # if y1 == y2 == 0.0 or x1==x2: --> rectangle of zero height or width
        if not(y1 == y2 == 0.0 or x1 == x2):
            if y1 == y2:  # rectangle
                moment = 0.5 * (x1 + x2)
                area = (x2 - x1) * y1
            elif y1 == 0.0 and y2 != 0.0:  # triangle, height y2
                moment = 2.0 / 3.0 * (x2-x1) + x1
                area = 0.5 * (x2 - x1) * y2
            elif y2 == 0.0 and y1 != 0.0:  # triangle, height y1
                moment = 1.0 / 3.0 * (x2 - x1) + x1
                area = 0.5 * (x2 - x1) * y1
            else:
                moment = (2.0 / 3.0 * (x2-x1) * (y2 + 0.5*y1)) / (y1+y2) + x1
                area = 0.5 * (x2 - x1) * (y1 + y2)

            sum_moment_area += moment * area
            sum_area += area

    return sum_moment_area / np.fmax(sum_area,
np.finfo(float).eps).astype(float)

In [5]:
volume = ctrl.Antecedent(np.arange(8, 21, 1), 'volume')
temperature = ctrl.Antecedent(np.arange(-18, 0, 1), 'temperature')
output = ctrl.Consequent(np.arange(0, 7, 1), 'output')

In [6]:
volume['small']=fuzz.trimf(volume.universe, [8, 9, 10])
volume['medium']=fuzz.trimf(volume.universe, [11, 13, 15])
volume['large']=fuzz.trimf(volume.universe, [15, 18, 21])

In [7]:
temperature['low']=fuzz.trimf(temperature.universe, [-18, -12, -9])
temperature['medium']=fuzz.trimf(temperature.universe, [-9, -6, -3])
temperature['high']=fuzz.trimf(temperature.universe, [-3, -1, 0])

In [8]:
output['low'] = fuzz.trimf(output.universe, [0, 1, 2])
output['medium'] = fuzz.trimf(output.universe, [2, 3, 5])
output['high'] = fuzz.trimf(output.universe, [5, 6, 7])

In [9]:
#volume['medium'].view()
#temperature.view()
#The higher number of knob means the colder environment of a refrigerator 
rule1 = ctrl.Rule(volume['small'] | temperature['low'], output['low'])
rule2 = ctrl.Rule(volume['medium'], output['medium'])
rule3 = ctrl.Rule(volume['large'] | temperature['high'], output['high'])
rule4 = ctrl.Rule(temperature['high'], output['high'])

In [10]:
tipping_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])

In [11]:
tipping = ctrl.ControlSystemSimulation(tipping_ctrl)

In [13]:
#input volume=16 cm cube
#input temperature= -2 degree celcius
tipping.input['volume'] = 16
tipping.input['temperature'] = -2

# Crunch the numbers
tipping.compute()

In [14]:
#defuzzification by COG
#tipping.defuzz()
print("Cool Knob Position is=",tipping.output['output'])
#output.view(sim=tipping)

Cool Knob Position is= 5.611111111111111
